In [ ]:
## load the libraries 
%matplotlib inline
import sys
import warnings
import os
import glob
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.metrics import *
from keras.optimizers import Adam, RMSprop
from scipy.stats import norm
from keras.preprocessing import image
from keras import datasets

from keras import backend as K
import random
import matplotlib.pyplot as plt
from skimage import exposure
plt.gray()

os.environ["CUDA_VISIBLE_DEVICES"]="0" #model will be trained on GPU 0

In [ ]:
def load_image(path):
    x_size = 64
    y_size = 64
    image_list = np.zeros((len(path), x_size, y_size,1))
    target_list = np.zeros((len(path), x_size, y_size,1))

    for i, fig in enumerate(path):
        img = image.load_img(fig, grayscale=True)#, target_size=(x_size, y_size))
        img = image.img_to_array(img).astype('float32')
        img = img / 255.0
        '''
        max_i = img.max()
        min_i = img.min()
        
        #print(img.shape)
        if max_i == min_i:
            img = np.zeros((64,64,1))
        else:
            img = (img - min_i) / (max_i - min_i)
            img = exposure.equalize_adapthist(img[:, :, 0])
        #print(img.shape)
        '''
        try:
            image_list[i] = img.reshape(64,64,1)#exposure.equalize_adapthist(img[:,:,0])
            
            target = cv2.imread(fig.split('.')[0] + '_target.png')[:, :, :1]
            target_list[i] = target
        except:
            print(fig)
            pass
        
    return image_list, target_list

In [ ]:
"""
train_img = glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/Plaque/Aug/cal/*.png")
train_img += glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/Plaque/Aug/fibrous/*.png")
train_img += random.sample(glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/NoPlaque/*.png"), 3790)
"""
all_img = [i for i in glob.glob("test4/*.png") if 'target' not in i]


all_noplaques = [i for i in all_img if 'noplaque' in i]
all_plaques = [i for i in all_img if 'noplaque' not in i]


all_noplaques_ids = list(set([os.path.basename(i).split('_')[1] for i in all_noplaques]))
_, all_noplaques_ids_test = train_test_split(all_noplaques_ids, test_size=30, random_state=42)

all_noplaques_ids_test = set(all_noplaques_ids_test)
all_noplaques = [i for i in all_noplaques if os.path.basename(i).split('_')[1] in all_noplaques_ids_test]
print(len(all_noplaques))
print(len(all_plaques))


all_img = all_noplaques + all_plaques
print(len(all_img))
ids = list(set([os.path.basename(i).split('_')[1] for i in all_img]))

ids_train, ids_test = train_test_split(ids, test_size=0.2, random_state=42)

ids_train = set(ids_train)
ids_test = set(ids_test)

train_img = [i for i in all_img if os.path.basename(i).split('_')[1] in ids_train]
test_img = [i for i in all_img if os.path.basename(i).split('_')[1] in ids_test]

random.shuffle(train_img)
random.shuffle(test_img)

x_train, y_train = load_image(train_img)
x_test, y_test = load_image(test_img) 



In [ ]:
img = x_train[10]
target = y_test[10]

In [ ]:
len(x_train), len(y_train), len(x_test), len(y_test)

In [ ]:
len(all_plaques), len(all_noplaques)

In [ ]:
set(train_img).intersection(set(test_img))

In [ ]:
print(img.shape)
plt.imshow(img[:,:,0])
plt.figure()
plt.imshow(target[:,:,0])

In [ ]:
x_test.shape, x_train.shape

plt.imshow(x_train[0, :, :, 0])

plt.figure()
plt.imshow(x_test[0, :, :, 0])

In [ ]:
np.unique(y_test), np.unique(y_train)

In [ ]:
batch_size = 128
epochs = 200
inChannel = 1
x, y = 64, 64
input_img = Input(shape = (x, y, inChannel))
print(input_img.shape)

In [ ]:
img_rows, img_cols, img_chns = x, y, 1
# number of convolutional filters to use
NB_FILTERS = 16
# convolution kernel size
KERNEL_SIZE = (3, 3)


LATENT_DIM = 128
INTERMEDIATE_DIM = 256
NB_EPOCHS = 100

conv_1 = Conv2D(1, kernel_size=(2, 2), padding='same', activation='relu')(input_img)
conv_2 = Conv2D(NB_FILTERS, kernel_size=(2, 2),
                       padding='same', activation='relu',
                       strides=(2, 2))(conv_1)
conv_3 = Conv2D(NB_FILTERS, KERNEL_SIZE,
                       padding='same', activation='relu',
                       strides=(1, 1))(conv_2)
conv_4 = Conv2D(NB_FILTERS, KERNEL_SIZE,
                       padding='same', activation='relu',
                       strides=(1, 1))(conv_3)
flat = Reshape((-1,))(conv_4)

dropout1 = Dropout(rate=0.4)(flat)
hidden = Dense(INTERMEDIATE_DIM, activation='relu')(dropout1)
dropout2 = Dropout(rate=0.4)(hidden)

z = Dense(LATENT_DIM)(dropout2)

decoder_hid = Dense(INTERMEDIATE_DIM, activation='relu')(z)

dropout3 = Dropout(rate=0.4)(decoder_hid)
decoder_upsample = Dense(NB_FILTERS * img_rows//2 * img_rows//2, activation='relu')(dropout3)
dropout4 = Dropout(rate=0.4)(decoder_upsample)

decoder_reshape = Reshape((img_rows//2, img_rows//2, NB_FILTERS))(dropout4)
decoder_deconv_1 = Conv2DTranspose(NB_FILTERS, KERNEL_SIZE,
                                   padding='same',
                                   strides=(1, 1),
                                   activation='relu')(decoder_reshape)
decoder_deconv_2 = Conv2DTranspose(NB_FILTERS, KERNEL_SIZE,
                                   padding='same',
                                   strides=(1, 1),
                                   activation='relu')(decoder_deconv_1)

decoder_deconv_3_upsamp = Conv2DTranspose(NB_FILTERS, kernel_size=(2, 2),
                                          padding='valid',
                                          strides=(2, 2),
                                          activation='relu')(decoder_deconv_2)
decoder_mean_squash = Conv2D(img_chns, kernel_size=(1, 1),
                                    padding='valid',
                                    activation='sigmoid')(decoder_deconv_3_upsamp)


In [ ]:
# compile the model

model = Model(input_img, decoder_mean_squash)
model.compile(optimizer=Adam(lr=0.0001), loss='mse')#dice_coef_loss, metrics=[dice_coef])
model.summary()

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
history = model.fit(
    x_train, 
    y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(x_test, y_test)
)#, callbacks=[early_stopping])

In [ ]:
plt.plot(history.history['loss'])
plt.title('Training Loss Plot')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

plt.title('Val Loss Plot')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
model.save('autoencoder.h5') 
model.save_weights('autoencoder_w.h5')

In [ ]:
# compile the model

# adam = optimizers.Adam(lr=0.01)

encoder = Model(input_img, z)
#encoder.compile(optimizer=Adam(lr=0.0001), loss='mse')#dice_coef, metrics=[dice_coef_loss])

In [ ]:
encoded_imgs_test = encoder.predict(x_test)
decoded_imgs_test = model.predict(x_test)

encoded_imgs_train = encoder.predict(x_train)
decoded_imgs_train = model.predict(x_train)

In [ ]:
encoded_imgs2 = encoder.predict(x2_test)
decoded_imgs2 = model.predict(x2_test)

In [ ]:
encoded_imgs.shape
#print(encoded_imgs[0].shape)

In [ ]:
encoded_imgs[0].shape

In [ ]:
label_conv = {
    0: 'NoPlaque',
    1: 'Calcific',
    2: 'Fibrous'
}

In [ ]:
distinct = [np.unique(y_train[i]) for i in range(y_train.shape[0])]
print([i for i in distinct if 1 in i and 2 in i])

distinct = [np.unique(y_test[i]) for i in range(y_test.shape[0])]
[i for i in distinct if 1 in i and 2 in i]

In [ ]:
x_set = x_train
y_set = y_train
encoded_imgs_set = encoded_imgs_train
decoded_imgs_set = decoded_imgs_train

for i in range(11):
    fig, ((ax1, ax2, ax3, ax4)) = plt.subplots(1, 4)
    
    label = y_set[i].max()
    fig.suptitle(f"[{i}] Label: " + label_conv[label], fontsize=16)
    
    ax1.imshow(x_set[i, :, :, 0])
    ax1.set_title("Original")
    
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    
    ax2.imshow(encoded_imgs_set[i].reshape((16, 8)))
    ax2.set_title("Encoded")
    
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    
    ax3.imshow(decoded_imgs_set[i].reshape((64, 64)))
    ax3.set_title("Decoded")
    
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    
    
    ax4.imshow(y_set[i].reshape((64, 64)))
    ax4.set_title("Target")
    
    ax4.get_xaxis().set_visible(False)
    ax4.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
x_set = x_train
y_set = y_train
encoded_imgs_set = encoded_imgs_train
decoded_imgs_set = decoded_imgs_train
i = train_img.index('test4/conv_D3TBFJS0_1.png')
fig, ((ax1, ax2, ax3, ax4)) = plt.subplots(1, 4)

label = y_set[i].max()
fig.suptitle(f"[{i}] Label: " + label_conv[label], fontsize=16)


ax1.imshow(x_set[i, :, :, 0])
ax1.set_title("Original")

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

ax2.imshow(encoded_imgs_set[i].reshape((16, 8)))
ax2.set_title("Encoded")
    
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)

ax3.imshow(decoded_imgs_set[i].reshape((64, 64)))
ax3.set_title("Decoded")

ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)


ax4.imshow(y_set[i].reshape((64, 64)))
ax4.set_title("Target")

ax4.get_xaxis().set_visible(False)
ax4.get_yaxis().set_visible(False)
plt.show()

In [ ]:
p = glob.glob('sagittal_conv_D538Q626*')


x3_test = load_image(p)

print(x3_test.shape)
y3_test = []


for img in p:
   
    if "Calcific" in img:
        y3_test.append(0)
    elif "Fibrous" in img:
        y3_test.append(1)
    elif "noplaque" in img:
        y3_test.append(2)

In [ ]:
encoded_imgs3 = encoder.predict(x3_test)
decoded_imgs3 = model.predict(x3_test)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(encoded_imgs[2].reshape(1,-1), encoded_imgs[1].reshape(1,-1))

In [ ]:
cos_sim

In [ ]:
np.sum(np.subtract(encoded_imgs[0], encoded_imgs[5]))

In [ ]:
#encoded_imgs[np.array(y_test) == 1]
plt.title("SCATTER")
plt.scatter(encoded_imgs[np.array(y_test) == 0][:, 0], encoded_imgs[np.array(y_test) == 0][:, 1], label='Calcific', alpha=0.5)
plt.scatter(encoded_imgs[np.array(y_test) == 1][:, 0], encoded_imgs[np.array(y_test) == 1][:, 1], label='Fibrous', alpha=0.5)
plt.scatter(encoded_imgs[np.array(y_test) == 2][:, 0], encoded_imgs[np.array(y_test) == 2][:, 1], label='No Plaque', alpha=0.5)
plt.scatter(encoded_imgs2[np.array(y2_test) == 2][:, 0], encoded_imgs2[np.array(y2_test) == 2][:, 1], label='Patch', alpha=0.5)
plt.legend();

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
proj = pca.fit_transform(encoded_imgs_train) 
#proj2 = pca.transform(encoded_imgs2)
#proj3 = pca.transform(encoded_imgs3)

In [ ]:
len(proj)

In [ ]:
len(pca.components_)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
proj[0]

In [ ]:
label_train = y_train.reshape(y_train.shape[0], -1).max(axis=1)
label_test = y_test.reshape(y_test.shape[0], -1).max(axis=1)

In [ ]:
label_train.shape

In [ ]:
plt.title("PCA")

plt.scatter(proj[np.array(label_train) == 1][:, 0], 
            proj[np.array(label_train) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 2][:, 0], 
            proj[np.array(label_train) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 0][:, 0], 
            proj[np.array(label_train) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
plt.title("PCA")

plt.scatter(proj[np.array(label_train) == 1][:, 0], 
            proj[np.array(label_train) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 2][:, 0], 
            proj[np.array(label_train) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 0][:, 0], 
            proj[np.array(label_train) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
plt.title("PCA")
plt.scatter(proj[np.array(y_test) == 0][:, 0], proj[np.array(y_test) == 0][:, 1], label='Calcific', alpha=0.5)
plt.scatter(proj[np.array(y_test) == 1][:, 0], proj[np.array(y_test) == 1][:, 1], label='Fibrous', alpha=0.5)
plt.scatter(proj[np.array(y_test) == 2][:, 0], proj[np.array(y_test) == 2][:, 1], label='No Plaque', alpha=0.5)

#plt.scatter(proj2[np.array(y2_test) == 0][:, 0], proj2[np.array(y2_test) == 0][:, 1], label='Patch Cal', alpha=0.5)
plt.scatter(proj2[np.array(y2_test) == 1][:, 0], proj2[np.array(y2_test) == 1][:, 1], label='Patch Fib', alpha=0.5)
#plt.scatter(proj2[np.array(y2_test) == 2][:, 0], proj2[np.array(y2_test) == 2][:, 1], label='Patch NoPl', alpha=0.5)


plt.scatter(proj3[np.array(y3_test) == 0][:, 0], proj3[np.array(y3_test) == 0][:, 1], label='New Cal', alpha=0.5)
plt.scatter(proj3[np.array(y3_test) == 1][:, 0], proj3[np.array(y3_test) == 1][:, 1], label='New Fib', alpha=0.5)
plt.scatter(proj3[np.array(y3_test) == 2][:, 0], proj3[np.array(y3_test) == 2][:, 1], label='New NoPl', alpha=0.5)


plt.legend();

In [ ]:
plt.scatter(proj3[np.array(y3_test) == 0][:, 0], proj3[np.array(y3_test) == 0][:, 1], label='New Cal', alpha=0.5)
plt.scatter(proj3[np.array(y3_test) == 1][:, 0], proj3[np.array(y3_test) == 1][:, 1], label='New Fib', alpha=0.5)
plt.scatter(proj3[np.array(y3_test) == 2][:, 0], proj3[np.array(y3_test) == 2][:, 1], label='New NoPl', alpha=0.5)
plt.legend();

In [ ]:
from umap import UMAP

umap = UMAP(random_state=42)
standard_embedding = umap.fit_transform(encoded_imgs)

standard_embedding2 = umap.transform(encoded_imgs2)
"""
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=y_test, cmap='Spectral', alpha=0.5)
plt.colorbar();
"""

In [ ]:
standard_embedding

In [ ]:
from umap import UMAP

umap = UMAP(random_state=42)
proj = umap.fit_transform(encoded_imgs_train)

"""
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=y_test, cmap='Spectral', alpha=0.5)
plt.colorbar();
"""

In [ ]:
plt.title("UMAP")

plt.scatter(proj[np.array(label_train) == 1][:, 0], 
            proj[np.array(label_train) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 2][:, 0], 
            proj[np.array(label_train) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 0][:, 0], 
            proj[np.array(label_train) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
plt.title("UMAP")

plt.scatter(proj[np.array(label_train) == 1][:, 0], 
            proj[np.array(label_train) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.figure()
plt.scatter(proj[np.array(label_train) == 2][:, 0], 
            proj[np.array(label_train) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.figure()
plt.scatter(proj[np.array(label_train) == 0][:, 0], 
            proj[np.array(label_train) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
y_train_mask = y_train.astype(np.bool)

y_train_mask_sum = y_train_mask.reshape(y_train_mask.shape[0], -1).sum(axis=1)

x_train_plaque_th = x_train[y_train_mask_sum > 500]
y_train_plaque_th = y_train[y_train_mask_sum > 500]
encoded_imgs_train_plaque_th = encoded_imgs_train[y_train_mask_sum > 500]


x_train_noplaque = x_train[y_train_mask_sum == 0]
y_train_noplaque = y_train[y_train_mask_sum == 0]
encoded_imgs_train_noplaque = encoded_imgs_train[y_train_mask_sum == 0]


y_train_sub = np.vstack((y_train_plaque_th, y_train_noplaque))
x_train_sub = np.vstack((x_train_plaque_th, x_train_noplaque))
encoded_imgs_train_sub = np.vstack((encoded_imgs_train_plaque_th, encoded_imgs_train_noplaque))

In [ ]:
y_test_mask = y_test.astype(np.bool)

y_test_mask_sum = y_test_mask.reshape(y_test_mask.shape[0], -1).sum(axis=1)

x_test_plaque_th = x_test[y_test_mask_sum > 500]
y_test_plaque_th = y_test[y_test_mask_sum > 500]
encoded_imgs_test_plaque_th = encoded_imgs_test[y_test_mask_sum > 500]


x_test_noplaque = x_test[y_test_mask_sum == 0]
y_test_noplaque = y_test[y_test_mask_sum == 0]
encoded_imgs_test_noplaque = encoded_imgs_test[y_test_mask_sum == 0]


y_test_sub = np.vstack((y_test_plaque_th, y_test_noplaque))
x_test_sub = np.vstack((x_test_plaque_th, x_test_noplaque))
encoded_imgs_test_sub = np.vstack((encoded_imgs_test_plaque_th, encoded_imgs_test_noplaque))

In [ ]:
from umap import UMAP

umap = UMAP(random_state=42)
proj = umap.fit_transform(encoded_imgs_train_sub)

"""
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=y_test, cmap='Spectral', alpha=0.5)
plt.colorbar();
"""

In [ ]:
label_train = y_train_sub.reshape(y_train_sub.shape[0], -1).max(axis=1)
label_test = y_test_sub.reshape(y_test_sub.shape[0], -1).max(axis=1)

In [ ]:



plt.figure(figsize=(8, 6), dpi=80)
plt.title("UMAP")
plt.scatter(proj[np.array(label_train) == 1][:, 0], 
            proj[np.array(label_train) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 2][:, 0], 
            proj[np.array(label_train) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.scatter(proj[np.array(label_train) == 0][:, 0], 
            proj[np.array(label_train) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
from umap import UMAP

umap = UMAP(random_state=42)
proj = umap.fit_transform(encoded_imgs_test_sub)

"""
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=y_test, cmap='Spectral', alpha=0.5)
plt.colorbar();
"""

In [ ]:
plt.title("UMAP")

plt.figure(figsize=(8, 6), dpi=80)
plt.scatter(proj[np.array(label_test) == 1][:, 0], 
            proj[np.array(label_test) == 1][:, 1], 
            label='Calcific', 
            alpha=0.5)

plt.scatter(proj[np.array(label_test) == 2][:, 0], 
            proj[np.array(label_test) == 2][:, 1], 
            label='Fibrous', 
            alpha=0.5)

plt.scatter(proj[np.array(label_test) == 0][:, 0], 
            proj[np.array(label_test) == 0][:, 1], 
            label='No Plaque', 
            alpha=0.5)

plt.legend()

In [ ]:
plt.title("UMAP")
plt.scatter(standard_embedding[np.array(y_test) == 0][:, 0], standard_embedding[np.array(y_test) == 0][:, 1], alpha=0.5, label='Calcific')
plt.scatter(standard_embedding[np.array(y_test) == 1][:, 0], standard_embedding[np.array(y_test) == 1][:, 1], alpha=0.5, label='Fibrous')
plt.scatter(standard_embedding[np.array(y_test) == 2][:, 0], standard_embedding[np.array(y_test) == 2][:, 1], alpha=0.5, label='NoPlaque')

plt.scatter(standard_embedding2[np.array(y2_test) == 0][:, 0], standard_embedding2[np.array(y2_test) == 0][:, 1], label='Patch Cal', alpha=0.5)
plt.scatter(standard_embedding2[np.array(y2_test) == 1][:, 0], standard_embedding2[np.array(y2_test) == 1][:, 1], label='Patch Fib', alpha=0.5)
#plt.scatter(standard_embedding2[np.array(y2_test) == 2][:, 0], standard_embedding2[np.array(y2_test) == 2][:, 1], label='Patch NoPl', alpha=0.5)

plt.legend();

In [ ]:
plt.scatter(standard_embedding2[np.array(y2_test) == 0][:, 0], standard_embedding2[np.array(y2_test) == 0][:, 1], label='Patch Cal', alpha=0.5)
plt.figure()
plt.scatter(standard_embedding2[np.array(y2_test) == 1][:, 0], standard_embedding2[np.array(y2_test) == 1][:, 1], label='Patch NoPl', alpha=0.5)

plt.figure()
plt.scatter(standard_embedding2[np.array(y2_test) == 2][:, 0], standard_embedding2[np.array(y2_test) == 2][:, 1], label='Patch Fib', alpha=0.5)


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
emb = tsne.fit_transform(encoded_imgs)

emb2 = tsne.transform(encoded_imgs2)

In [ ]:
plt.title("T-SNE")
plt.scatter(emb[np.array(y_test) == 0][:, 0], tsne[np.array(y_test) == 0][:, 1], alpha=0.5, label='Calcific')
plt.scatter(emb[np.array(y_test) == 1][:, 0], tsne[np.array(y_test) == 1][:, 1], alpha=0.5, label='Fibrous')
plt.scatter(emb[np.array(y_test) == 2][:, 0], tsne[np.array(y_test) == 2][:, 1], alpha=0.5, label='NoPlaque')

plt.scatter(emb2[np.array(y2_test) == 0][:, 0], emb2[np.array(y2_test) == 0][:, 1], label='Patch', alpha=0.5)
plt.scatter(emb2[np.array(y2_test) == 1][:, 0], emb2[np.array(y2_test) == 1][:, 1], label='Patch', alpha=0.5)
plt.scatter(emb2[np.array(y2_test) == 2][:, 0], emb2[np.array(y2_test) == 2][:, 1], label='Patch', alpha=0.5)

plt.legend();

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components=2).fit_transform(encoded_imgs)

In [ ]:
plt.title("MDS")
plt.scatter(mds[np.array(y_test) == 0][:, 0], mds[np.array(y_test) == 0][:, 1], alpha=0.5, label='Calcific')
plt.scatter(mds[np.array(y_test) == 1][:, 0], mds[np.array(y_test) == 1][:, 1], alpha=0.5, label='Fibrous')
plt.scatter(mds[np.array(y_test) == 2][:, 0], mds[np.array(y_test) == 2][:, 1], alpha=0.5, label='NoPlaque')
plt.legend();